In [13]:
import torch 
from torch import nn
from torch.optim import Adam
from torch.nn import CrossEntropyLoss, Linear, ReLU, Sequential
import cv2
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
from torchvision import transforms

In [14]:
DATA_PATH = "../data/"
hyperparameters = {
    'batch_size': 32,
    'learning_rate': 0.001,
    'epochs': 10,
}

In [15]:
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, csv_file, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.img_labels = pd.read_csv(csv_file)
    def __len__(self):
        return len(self.img_labels)
    def __getitem__(self, index):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[index, 0])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.img_labels.iloc[index, 1]
        if self.transform:
            image = self.transform(image)
        return (image, label)    

In [16]:
class TestImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_id = self.df.iloc[index, 0]
        img_path = os.path.join(self.img_dir, img_id)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        return (image, img_id)


In [17]:
original_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

train_dataset = CustomImageDataset(img_dir=DATA_PATH + "train_images", csv_file=DATA_PATH + "train.csv", transform=original_transform)
val_dataset = CustomImageDataset(img_dir=DATA_PATH + "val_images", csv_file=DATA_PATH + "val.csv", transform=original_transform)
test_dataset = TestImageDataset(img_dir=DATA_PATH + "test_images", csv_file=DATA_PATH + "test.csv", transform=original_transform)

train_loader = DataLoader(train_dataset, batch_size=hyperparameters['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=hyperparameters['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=hyperparameters['batch_size'], shuffle=False)

In [18]:
# print first image and label
print(train_dataset[1])
print(val_dataset[1])

(tensor([[[0.4471, 0.5020, 0.5961,  ..., 0.3686, 0.4471, 0.4471],
         [0.4745, 0.5294, 0.5176,  ..., 0.3608, 0.4314, 0.4510],
         [0.4745, 0.5333, 0.4588,  ..., 0.3686, 0.2863, 0.4235],
         ...,
         [0.4588, 0.5020, 0.5098,  ..., 0.2706, 0.4471, 0.4588],
         [0.4588, 0.5961, 0.4667,  ..., 0.2941, 0.3765, 0.4392],
         [0.4902, 0.5843, 0.5843,  ..., 0.4627, 0.4863, 0.4745]],

        [[0.4588, 0.5333, 0.6157,  ..., 0.4039, 0.4667, 0.4314],
         [0.4745, 0.5608, 0.5373,  ..., 0.3882, 0.4510, 0.4275],
         [0.4745, 0.5686, 0.4902,  ..., 0.3843, 0.3137, 0.4118],
         ...,
         [0.4549, 0.5176, 0.5529,  ..., 0.2824, 0.4588, 0.4510],
         [0.4627, 0.6314, 0.4980,  ..., 0.3137, 0.4000, 0.4314],
         [0.4824, 0.6196, 0.6235,  ..., 0.4941, 0.5137, 0.4706]],

        [[0.3882, 0.4510, 0.4863,  ..., 0.3373, 0.3961, 0.3451],
         [0.4157, 0.4667, 0.4039,  ..., 0.3255, 0.3804, 0.3451],
         [0.4157, 0.4745, 0.3882,  ..., 0.3373, 0.2784, 0

In [7]:
print("Pytorch CUDA Version is ", torch.version.cuda)
print("Whether CUDA is supported by our system:", torch.cuda.is_available())

Pytorch CUDA Version is  11.7
Whether CUDA is supported by our system: False


In [19]:
class Net(nn.Module):
    def __init__(self, num_classes=96):
        super(Net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classsifier = nn.Sequential(
            nn.Linear(512*2*2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),

            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),

            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch, you can also use x.view(x.size(0), -1)
        x = self.classsifier(x)
        return x
    

            

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=hyperparameters['learning_rate'])
num_epochs = hyperparameters['epochs']

for epoch in range(num_epochs):
    # set the model to train mode
    # enable dropout, batch normalization etc.
    model.train()
    for images, labels in train_loader:
        # loads the images to cuda if availabl
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images) # forward pass
        loss = criterion(outputs, labels) # compute the loss

        optimizer.zero_grad() # reset the gradients because they accumulate by default
        loss.backward() # compute the gradients in the backward pass
        optimizer.step() # update the parameters based on the gradients computed in the backward pass

    # set the model to evaluation mode
    # disable dropout, batch normalization etc.
    model.eval()
    with torch.no_grad(): # to disable gradient calculation and backpropagation
        correct = 0
        total = 0
        for images, labels in val_loader:
            # loads the images to cuda if availabl
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images) # forward pass
            # torch.max returns a tuple (values, indices) where indices is the index of the maximum value of a tensor along a dimension
            _, predicted = torch.max(outputs.data, 1) # get the predicted class with highest probability
            total += labels.size(0) # total number of labels in a batch
            correct += (predicted == labels).sum().item() # total correct predictions

        print('Epoch [{}/{}], Validation Accuracy: {:.2f}%'
              .format(epoch+1, num_epochs, 100 * correct / total))

        

Epoch [1/10], Validation Accuracy: 3.60%


KeyboardInterrupt: 

In [10]:
# test torch.max
# a = torch.randn(4, 4)
# print(a)

# x, y = torch.max(a, 0)
# x, y = torch.max(a, 1)
# print(x, y)

#test labels.size(0) and convert tensor to int with .item()

# for images, labels in train_loader:
#     print(labels, images)  
#     print(labels.size(0))
#     # create a tensor of size labels.size(0) with values from 1 to 96 and compare it with labels
#     predicted = torch.tensor([i for i in range(1, labels.size(0)+1)])
#     print(predicted)
#     print((predicted == labels).sum().item())
#     break
    

In [ ]:
model.eval()

predicted_labels = []
with torch.no_grad(): # to disable gradient calculation and backpropagation
    for images, labels in test_loader:
        images = images.to(device)

        outputs = model(images) # forward pass

        _, predicted = torch.max(outputs.data, 1) # get the predicted class with highest probability

        predicted_labels.extend(predicted.tolist())

In [12]:
def save_model(model, optimizer, path):
    state = {'model': model.state_dict(),
             'optimizer': optimizer.state_dict()}
    torch.save(state, path)

# def load_model(model, optimizer, path):
#     state = torch.load(path)
#     model.load_state_dict(state['model'])
#     optimizer.load_state_dict(state['optimizer'])
#     return model, optimizer
def load_model(path):
    state = torch.load(path)
    return state['model']

In [12]:
# save_model(model, optimizer, '../models/model.pth')

In [13]:
def test_model(model, test_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # model.eval()
    predicted_labels = []
    with torch.no_grad(): # to disable gradient calculation and backpropagation
        for images in test_loader:
            images = images.to(device)

            outputs = model(images) # forward pass

            _, predicted = torch.max(outputs.data, 1) # get the predicted class with highest probability

            predicted_labels.extend(predicted.tolist())
    return predicted_labels

In [17]:
# model = load_model('../models/model.pth')
predicted_labels = test_model(model, test_loader)

AttributeError: 'list' object has no attribute 'to'

In [ ]:
import pickle

with open("tester.txt", "rb") as fp:   # Unpickling
    tester = pickle.load(fp)

print(tester)